In [1]:
# # LO17 - TD2 & TD3 : Préparation et Filtrage du Corpus ADIT
#
# Ce notebook met en œuvre les étapes décrites dans les TD2 et TD3 en utilisant les classes Python créées.
#
# **Objectifs :**
# 1.  **TD2 : Préparation du Corpus**
#     *   Parser les fichiers HTML des bulletins ADIT.
#     *   Extraire les informations structurées (métadonnées, texte, images, etc.).
#     *   Générer un fichier XML unique (`corpus_initial.xml`) contenant tous les articles parsés.
# 2.  **TD3 : Anti-dictionnaire et Filtrage**
#     *   Définir l'unité documentaire (ici, l'article/bulletin individuel).
#     *   Segmenter le corpus en tokens (`token`, `document_id`).
#     *   Calculer TF, IDF et TF-IDF.
#     *   Générer un anti-dictionnaire basé sur un seuil IDF.
#     *   Appliquer les substitutions (suppressions) de l'anti-dictionnaire au corpus en mémoire.
#     *   Sauvegarder le corpus filtré dans un nouveau fichier XML (`corpus_filtre.xml`).

# ## 1. Importations et Configuration

# +
import os
import pandas as pd
import warnings
from index import BS4Parser, Corpus


# Configuration des chemins
BASE_DIR = os.getcwd() # Ou spécifiez un chemin absolu
DATA_FOLDER = os.path.join(BASE_DIR, "data", "BULLETINS")
OUTPUT_FOLDER = os.path.join(BASE_DIR, "output")
XML_INITIAL_FILE = os.path.join(OUTPUT_FOLDER, "corpus_initial.xml")
XML_FILTRE_FILE = os.path.join(OUTPUT_FOLDER, "corpus_filtre.xml")
SEGMENTATION_FILE = os.path.join(OUTPUT_FOLDER, "segmentation.tsv")
TF_FILE = os.path.join(OUTPUT_FOLDER, "tf.tsv")
IDF_FILE = os.path.join(OUTPUT_FOLDER, "idf.tsv")
TFIDF_FILE = os.path.join(OUTPUT_FOLDER, "tfidf.tsv")
ANTI_DICT_FILE = os.path.join(OUTPUT_FOLDER, "anti_dictionnaire.txt")

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Configuration pour pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 150)

# Ignorer certains warnings de BeautifulSoup si nécessaire
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

print(f"Dossier des données (BULLETINS): {DATA_FOLDER}")
print(f"Dossier de sortie: {OUTPUT_FOLDER}")
# -

KeyboardInterrupt: 

In [ ]:
# ## 2. TD2 - Chargement et Structuration du Corpus

# ### 2.1. Instanciation du Parser
#
# Nous utilisons `BS4Parser` qui hérite de `FileProcessClient` pour lire et traiter les fichiers HTML.

# +
# Instancier le parser spécifique pour les fichiers HTML ADIT
html_parser = BS4Parser()
print("Parser BS4Parser instancié.")
# -

# ### 2.2. Chargement des Documents depuis le Dossier
#
# `Corpus.from_folder` utilise le `BS4Parser` pour traiter tous les fichiers `.htm` dans le dossier spécifié.

# +
print(f"Chargement des documents depuis {DATA_FOLDER}...")
# Utiliser Corpus.from_folder avec le parser
# Pas de limite par défaut, charge tous les fichiers
try:
    corpus = Corpus.from_folder(html_parser, DATA_FOLDER)
    print(f"Chargement terminé. {len(corpus.documents)} documents ont été chargés et parsés.")
except FileNotFoundError:
    print(f"ERREUR: Le dossier '{DATA_FOLDER}' n'a pas été trouvé.")
    print("Vérifiez que le chemin est correct et que les données sont présentes.")
    # Stopper l'exécution ou gérer l'erreur autrement
    corpus = None
except TypeError as e:
    print(f"ERREUR: Problème de type lors du chargement: {e}")
    corpus = None
except Exception as e:
    print(f"ERREUR inattendue lors du chargement: {e}")
    corpus = None

# Afficher un exemple de document chargé (si le chargement a réussi)
if corpus and corpus.documents:
    print("\nExemple de document chargé (premier document) :")
    # Utilise la représentation Pydantic par défaut, qui est assez lisible
    print(corpus.documents[0].model_dump(exclude_none=True))
else:
    print("\nAucun document n'a pu être chargé. Vérifiez les erreurs précédentes.")
# -

In [ ]:
corpus.save_to_xml()

In [ ]:
# ### 2.3. Sauvegarde du Corpus Initial en XML
#
# La méthode `save_to_xml` du `Corpus` sérialise chaque objet `Document` (qui hérite de `XMLBaseModel`) en XML et les regroupe sous une balise racine `<Corpus>`.

# +
if corpus and corpus.documents:
    print(f"\nSauvegarde du corpus initial dans {XML_INITIAL_FILE}...")
    corpus.save_to_xml(XML_INITIAL_FILE, root_tag="corpus") # TD demande <corpus>

    # Note importante sur la structure XML générée :
    # Par défaut, XMLBaseModel utilise le nom de la classe comme balise pour chaque document.
    # Ici, ce sera <Document>. Le TD demandait <bulletin>.
    # Pour obtenir <bulletin>, il faudrait soit:
    # 1. Modifier la classe Document pour surcharger `model_dump_xml` et forcer le tag 'bulletin'.
    #    Exemple (dans la classe Document):
    #    def model_dump_xml(self, tag: Optional[str] = None) -> ET.Element:
    #        return super().model_dump_xml(tag="bulletin")
    # 2. Modifier la méthode `Corpus.save_to_xml` pour passer tag="bulletin" à doc.model_dump_xml().
    # Pour ce notebook, nous utilisons le code tel quel et générons des balises <Document>.

    print(f"\nContenu du début du fichier XML généré ({XML_INITIAL_FILE}):")
    try:
        with open(XML_INITIAL_FILE, 'r', encoding='utf-8') as f:
            for _ in range(15): # Afficher les 15 premières lignes
                line = f.readline()
                if not line: break
                print(line, end='')
    except FileNotFoundError:
        print(f"Erreur: Le fichier {XML_INITIAL_FILE} n'a pas été trouvé après tentative de sauvegarde.")

else:
    print("\nSauvegarde XML annulée car aucun document n'a été chargé.")
# -


Sauvegarde du corpus initial dans C:\Users\Raphael\Documents\TD02\output\corpus_initial.xml...
Sauvegarde du corpus actuel (326 documents) dans C:\Users\Raphael\Documents\TD02\output\corpus_initial.xml...
Corpus sauvegardé avec succès dans 'C:\Users\Raphael\Documents\TD02\output\corpus_initial.xml'.

Contenu du début du fichier XML généré (C:\Users\Raphael\Documents\TD02\output\corpus_initial.xml):
<?xml version="1.0" encoding="utf-8"?>
<corpus>
  <bulletin>
    <fichier>67068.htm</fichier>
    <numero>258</numero>
    <date>21/06/2011</date>
    <rubrique>Focus</rubrique>
    <titre>Physique : Mathias Fink, un bel exemple de chercheur qui innove</titre>
    <auteur>ADIT - Jean-François Desessard - email : jfd@adit.fr</auteur>
    <contact>Institut Langevin &quot;Ondes et Images&quot; - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr</contact>
    <texte>Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l'Innovation, dont 

In [ ]:
# ## 3. TD3 - Calcul TF-IDF et Filtrage

# ### 3.1. Choix de l'Unité Documentaire
#
# Le TD demande de réfléchir au choix de l'unité documentaire (bulletin entier vs article).
#
# **Notre approche actuelle :**
# *   Le `BS4Parser` crée un objet `Document` pour chaque fichier `.htm`. Chaque fichier semble correspondre à un *article* ou une *page* spécifique d'un bulletin.
# *   Le `Corpus` contient une liste de ces objets `Document`.
# *   Le `CorpusTFIDFCalculator` travaille sur cette liste de `Document`.
#
# **Conclusion :** L'unité documentaire implicitement utilisée par notre code est **l'article individuel** (représenté par un objet `Document` et identifié par son nom de fichier `fichier`). Le TF est calculé par article, et l'IDF est basé sur le nombre d'articles contenant un mot donné.

# ### 3.2. Instanciation du Calculateur TF-IDF
#
# Nous créons une instance de `CorpusTFIDFCalculator` en lui passant notre `Corpus`.

# +
if corpus and corpus.documents:
    print("Instanciation de CorpusTFIDFCalculator...")
    try:
        # 'fichier' est le champ utilisé pour l'ID du document dans la classe Document
        tfidf_calculator = CorpusTFIDFCalculator(corpus, doc_id_field='fichier')
        print("CorpusTFIDFCalculator instancié avec succès.")
    except ValueError as e:
        print(f"ERREUR lors de l'instanciation du calculateur TF-IDF: {e}")
        tfidf_calculator = None
    except Exception as e:
        print(f"ERREUR inattendue lors de l'instanciation: {e}")
        tfidf_calculator = None
else:
    print("\nImpossible d'instancier CorpusTFIDFCalculator car le corpus n'est pas chargé.")
    tfidf_calculator = None
# -

Instanciation de CorpusTFIDFCalculator...
CorpusTFIDFCalculator instancié avec succès.


In [ ]:
# ### 3.3. Segmentation (Équivalent `segmente.py`)
#
# La méthode `segmente()` du `Corpus` extrait tous les tokens (`mot`, `document_id`) du corpus. Le TD3 demandait un script `segmente.py` produisant un fichier tabulé. Nous allons générer un DataFrame puis l'enregistrer dans ce format.

# +
if corpus and corpus.documents:
    print("\nExécution de la segmentation (Corpus.segmente)...")
    segmentation_df = corpus.segmente()

    print(f"\nSegmentation terminée. {len(segmentation_df)} paires (token, document_id) trouvées.")
    print("Aperçu du DataFrame de segmentation :")
    display(segmentation_df.head())

    # Sauvegarde au format TSV (comme demandé pour segmente.py)
    print(f"\nSauvegarde de la segmentation dans {SEGMENTATION_FILE}...")
    try:
        # Pas d'index, pas d'en-tête, séparateur tabulation
        segmentation_df.to_csv(SEGMENTATION_FILE, sep='\t', index=False, header=False, encoding='utf-8')
        print("Sauvegarde terminée.")

        # Afficher le début du fichier généré
        print(f"\nContenu du début du fichier {SEGMENTATION_FILE}:")
        try:
            with open(SEGMENTATION_FILE, 'r', encoding='utf-8') as f:
                for _ in range(10):
                    line = f.readline()
                    if not line: break
                    print(line, end='')
        except FileNotFoundError:
             print(f"Erreur: Le fichier {SEGMENTATION_FILE} n'a pas été trouvé après tentative de sauvegarde.")

    except Exception as e:
        print(f"Erreur lors de la sauvegarde de la segmentation: {e}")

else:
    print("\nSegmentation annulée car le corpus n'est pas chargé.")
    segmentation_df = pd.DataFrame(columns=['token', 'document_id']) # DataFrame vide
# -


Exécution de la segmentation (CorpusClient.segmente)...

Segmentation terminée. 166183 paires (token, document_id) trouvées.
Aperçu du DataFrame de segmentation :


,token,document_id
0,physique,67068.htm
1,mathias,67068.htm
2,fink,67068.htm
3,un,67068.htm
4,bel,67068.htm



Sauvegarde de la segmentation dans C:\Users\Raphael\Documents\TD02\output\segmentation.tsv...
Sauvegarde terminée.

Contenu du début du fichier C:\Users\Raphael\Documents\TD02\output\segmentation.tsv:
physique	67068.htm
mathias	67068.htm
fink	67068.htm
un	67068.htm
bel	67068.htm
exemple	67068.htm
de	67068.htm
chercheur	67068.htm
qui	67068.htm
innove	67068.htm


In [ ]:
# +
if tfidf_calculator:
    # --- Calcul TF ---
    print("\nCalcul de TF (Term Frequency)...")
    tf_df = tfidf_calculator.calculate_tf()
    print("Aperçu du DataFrame TF :")
    display(tf_df.head())
    # Sauvegarde optionnelle du TF (format demandé par TD3)
    if not tf_df.empty:
        print(f"Sauvegarde de TF dans {TF_FILE}...")
        try:
            # Colonnes: identifiant_du_document, mot, tf
            tf_df.to_csv(TF_FILE, sep='\t', index=False, header=True, encoding='utf-8')
            print("Sauvegarde TF terminée.")
        except Exception as e:
            print(f"Erreur lors de la sauvegarde de TF: {e}")
    else:
        print("DataFrame TF est vide, sauvegarde annulée.")

    # --- Calcul IDF ---
    print("\nCalcul de IDF (Inverse Document Frequency)...")
    idf_df = tfidf_calculator.calculate_idf()
    print("Aperçu du DataFrame IDF :")
    # Afficher les mots avec IDF le plus bas (les plus fréquents/communs)
    display(idf_df.sort_values(by='idf').head())
    # Afficher les mots avec IDF le plus haut (les plus rares/discriminants)
    #display(idf_df.sort_values(by='idf', ascending=False).head())

    # Sauvegarde optionnelle de l'IDF (format demandé par TD3)
    if not idf_df.empty:
        print(f"Sauvegarde de IDF dans {IDF_FILE}...")
        try:
            # Colonnes: mot, idf
            idf_df.to_csv(IDF_FILE, sep='\t', index=False, header=True, encoding='utf-8')
            print("Sauvegarde IDF terminée.")
        except Exception as e:
            print(f"Erreur lors de la sauvegarde de IDF: {e}")
    else:
        print("DataFrame IDF est vide, sauvegarde annulée.")

    # --- Calcul TF-IDF ---
    print("\nCalcul de TF-IDF...")
    tfidf_df = tfidf_calculator.calculate_tfidf()
    print("Aperçu du DataFrame TF-IDF (trié par score décroissant pour le premier document):")
    if not tfidf_df.empty:
        first_doc_id = tfidf_df['document_id'].iloc[0]
        display(tfidf_df[tfidf_df['document_id'] == first_doc_id].sort_values(by='tf_idf', ascending=False).head(10))
    else:
        display(tfidf_df.head()) # Affichera un DataFrame vide avec les bonnes colonnes

    # Sauvegarde optionnelle du TF-IDF (format demandé par TD3)
    if not tfidf_df.empty:
        print(f"Sauvegarde de TF-IDF dans {TFIDF_FILE}...")
        try:
            # Colonnes: identifiant_du_document, mot, tf_idf
            tfidf_df.to_csv(TFIDF_FILE, sep='\t', index=False, header=True, encoding='utf-8')
            print("Sauvegarde TF-IDF terminée.")
        except Exception as e:
            print(f"Erreur lors de la sauvegarde de TF-IDF: {e}")
    else:
        print("DataFrame TF-IDF est vide, sauvegarde annulée.")

else:
    print("\nCalculs TF/IDF/TF-IDF annulés car le calculateur n'a pas été instancié.")
    tf_df = pd.DataFrame(columns=['document_id', 'mot', 'tf'])
    idf_df = pd.DataFrame(columns=['mot', 'idf'])
    tfidf_df = pd.DataFrame(columns=['document_id', 'mot', 'tf_idf'])

# -


Calcul de TF (Term Frequency)...
Recalcul de TF...
Calcul TF terminé. 80673 entrées TF générées.
Aperçu du DataFrame TF :


,document_id,mot,tf
0,67068.htm,physique,10
1,67068.htm,mathias,10
2,67068.htm,fink,10
3,67068.htm,un,13
4,67068.htm,bel,1


Sauvegarde de TF dans C:\Users\Raphael\Documents\TD02\output\tf.tsv...
Sauvegarde TF terminée.

Calcul de IDF (Inverse Document Frequency)...
Recalcul de IDF...
Calcul IDF terminé. 14486 mots uniques avec IDF calculé.
Aperçu du DataFrame IDF :


,mot,idf
7427,l,0.000000
14107,à,0.000000
3720,de,0.000000
5069,et,0.000000
7428,la,0.001334


Sauvegarde de IDF dans C:\Users\Raphael\Documents\TD02\output\idf.tsv...
Sauvegarde IDF terminée.

Calcul de TF-IDF...
Recalcul de TF-IDF...
Calcul TF-IDF terminé. 80673 entrées TF-IDF générées.
Aperçu du DataFrame TF-IDF (trié par score décroissant pour le premier document):


C:\Users\Raphael\Documents\TD02\index\clients\corpus_client.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tfidf_df['idf'].fillna(0, inplace=True)


,document_id,mot,tf_idf
2,67068.htm,fink,25.132176
1,67068.htm,mathias,25.132176
73,67068.htm,ondes,14.718249
0,67068.htm,physique,11.514898
162,67068.htm,renversement,10.052870
86,67068.htm,espci,9.071238
140,67068.htm,charpak,7.539653
74,67068.htm,images,6.685632
72,67068.htm,langevin,6.636563
210,67068.htm,vous,6.584172


Sauvegarde de TF-IDF dans C:\Users\Raphael\Documents\TD02\output\tfidf.tsv...
Sauvegarde TF-IDF terminée.


In [ ]:
# ### 3.4. Calcul TF, IDF, TF-IDF
#
# Nous utilisons les méthodes du `CorpusTFIDFCalculator`. Les résultats sont mis en cache dans l'instance du calculateur.


# ### 3.5. Génération de l'Anti-dictionnaire
#
# L'anti-dictionnaire contient les mots considérés comme non significatifs, basé sur un seuil IDF bas. Un IDF faible signifie que le mot apparaît dans de nombreux documents.

# +
if tfidf_calculator:
    # Choix du seuil IDF
    # Un seuil bas inclura plus de mots (ceux présents dans beaucoup de documents).
    # Le TD suggère 0.1, mais on peut expérimenter.
    IDF_THRESHOLD = 0.05 # Essayons un seuil un peu plus bas pour capturer les mots très communs

    # La méthode generate_anti_dict utilise le idf_df calculé précédemment
    generated_anti_dict_path = tfidf_calculator.generate_anti_dict(
        idf_threshold=IDF_THRESHOLD,
        output_file=ANTI_DICT_FILE
    )

    print(f"\nChemin du fichier généré : {generated_anti_dict_path}")

    # Afficher le début du fichier anti-dictionnaire
    print(f"Contenu du début du fichier {ANTI_DICT_FILE}:")
    try:
        with open(ANTI_DICT_FILE, 'r', encoding='utf-8') as f:
            for _ in range(20): # Afficher les 20 premiers mots
                line = f.readline()
                if not line: break
                print(line, end='')
    except FileNotFoundError:
        print(f"Erreur: Le fichier {ANTI_DICT_FILE} n'a pas été trouvé après tentative de génération.")

else:
    print("\nGénération de l'anti-dictionnaire annulée car le calculateur n'a pas été instancié.")
# -

Génération de l'anti-dictionnaire avec seuil IDF <= 0.05...
Nombre de mots candidats pour l'anti-dictionnaire: 16
Exemples de mots (les 20 avec le plus bas IDF parmi les candidats):
        mot       idf
3720     de  0.000000
7427      l  0.000000
5069     et  0.000000
14107     à  0.000000
7428     la  0.001334
7564     le  0.002673
4741     en  0.005362
3787    des  0.005362
7601    les  0.005362
3683      d  0.008068
10211  pour  0.021856
4172     du  0.023259
13529   une  0.031775
13527    un  0.033211
10897   qui  0.043396
3696   dans  0.049325
Anti-dictionnaire sauvegardé dans 'C:\Users\Raphael\Documents\TD02\output\anti_dictionnaire.txt'

Chemin du fichier généré : C:\Users\Raphael\Documents\TD02\output\anti_dictionnaire.txt
Contenu du début du fichier C:\Users\Raphael\Documents\TD02\output\anti_dictionnaire.txt:
d	""
dans	""
de	""
des	""
du	""
en	""
et	""
l	""
la	""
le	""
les	""
pour	""
qui	""
un	""
une	""
à	""


In [ ]:
# ### 3.6. Application des Substitutions (Filtrage)
#
# La méthode `apply_substitutions` du `Corpus` lit l'anti-dictionnaire et modifie *en mémoire* les champs texte (`titre`, `texte` comme défini dans `corps_fields` de `Document`) des objets `Document`. Elle invalide aussi le cache de la propriété `corps`.

# +
if corpus and os.path.exists(ANTI_DICT_FILE):
    print(f"\nApplication des substitutions depuis {ANTI_DICT_FILE} sur le corpus en mémoire...")

    # Attention: Cette opération modifie les objets Document dans corpus.documents
    corpus.apply_substitutions(ANTI_DICT_FILE)

    # Vérification (Optionnel) : Afficher le 'corps' d'un document avant et après
    # Note : Pour que cela fonctionne, il faudrait garder une copie du document original
    # ou recharger un document spécifique.
    # Ici, on peut juste ré-afficher un document pour voir s'il semble modifié.
    if corpus.documents:
        print("\nExemple de document après substitutions (premier document) :")
        # Afficher le corps recalculé (le cache a été invalidé si modifié)
        print("Nouveau 'corps' recalculé :")
        print(corpus.documents[0].corps[:500] + "...") # Afficher les 500 premiers caractères

elif not corpus:
    print("\nApplication des substitutions annulée car le corpus n'est pas chargé.")
elif not os.path.exists(ANTI_DICT_FILE):
    print(f"\nApplication des substitutions annulée car le fichier {ANTI_DICT_FILE} n'existe pas.")

# -


Application des substitutions depuis C:\Users\Raphael\Documents\TD02\output\anti_dictionnaire.txt sur le corpus en mémoire...
Application des substitutions depuis C:\Users\Raphael\Documents\TD02\output\anti_dictionnaire.txt sur 326 documents...
  ... 100/326 documents traités.
  ... 200/326 documents traités.
  ... 300/326 documents traités.
  ... 326/326 documents traités.
Substitutions appliquées. 326 documents ont été modifiés.
Le cache de la propriété 'corps' a été invalidé pour les documents modifiés.

Exemple de document après substitutions (premier document) :
Nouveau 'corps' recalculé :
Physique : Mathias Fink, bel exemple chercheur innove
27 avril dernier, CNRS décernait première fois Médaille Innovation, dont Valérie Pécresse, ministre Enseignement Supérieur Recherche est origine création. CNRS souhaite ainsi honorer chercheurs ingénieurs travaillant établissements publics recherche, universités, grandes écoles mais aussi industriels développent innovations marquantes. cette

In [ ]:

# ### 3.7. Sauvegarde du Corpus Filtré en XML
#
# Nous sauvegardons à nouveau le corpus, mais cette fois les objets `Document` ont été modifiés en mémoire par `apply_substitutions`.

# +
if corpus and corpus.documents:
    print(f"\nSauvegarde du corpus filtré (après substitutions) dans {XML_FILTRE_FILE}...")
    corpus.save_to_xml(XML_FILTRE_FILE, root_tag="corpus") # TD demande <corpus>

    print(f"\nCorpus filtré sauvegardé. Vérifiez le fichier {XML_FILTRE_FILE}.")

    # Afficher le début du fichier XML filtré
    print(f"\nContenu du début du fichier XML filtré ({XML_FILTRE_FILE}):")
    try:
        with open(XML_FILTRE_FILE, 'r', encoding='utf-8') as f:
            for _ in range(20): # Afficher les 20 premières lignes
                line = f.readline()
                if not line: break
                print(line, end='')
    except FileNotFoundError:
        print(f"Erreur: Le fichier {XML_FILTRE_FILE} n'a pas été trouvé après tentative de sauvegarde.")

else:
    print("\nSauvegarde du corpus filtré annulée car le corpus n'est pas chargé ou n'a pas été modifié.")
# -

# ## 4. Conclusion
#
# Le processus est terminé.
#
# **Fichiers générés dans le dossier `output/` :**
# *   `corpus_initial.xml`: Le corpus original structuré en XML.
# *   `segmentation.tsv`: Liste des paires (token, document_id).
# *   `tf.tsv`: Fréquence des termes par document.
# *   `idf.tsv`: Inverse Document Frequency pour chaque terme unique.
# *   `tfidf.tsv`: Score TF-IDF pour chaque terme dans chaque document.
# *   `anti_dictionnaire.txt`: Liste des mots à supprimer (basée sur le seuil IDF).
# *   `corpus_filtre.xml`: Le corpus après suppression des mots de l'anti-dictionnaire.


Sauvegarde du corpus filtré (après substitutions) dans C:\Users\Raphael\Documents\TD02\output\corpus_filtre.xml...
Sauvegarde du corpus actuel (326 documents) dans C:\Users\Raphael\Documents\TD02\output\corpus_filtre.xml...
Corpus sauvegardé avec succès dans 'C:\Users\Raphael\Documents\TD02\output\corpus_filtre.xml'.

Corpus filtré sauvegardé. Vérifiez le fichier C:\Users\Raphael\Documents\TD02\output\corpus_filtre.xml.

Contenu du début du fichier XML filtré (C:\Users\Raphael\Documents\TD02\output\corpus_filtre.xml):
<?xml version="1.0" encoding="utf-8"?>
<corpus>
  <bulletin>
    <fichier>67068.htm</fichier>
    <numero>258</numero>
    <date>21/06/2011</date>
    <rubrique>Focus</rubrique>
    <titre>Physique : Mathias Fink, bel exemple chercheur innove</titre>
    <auteur>ADIT - Jean-François Desessard - email : jfd@adit.fr</auteur>
    <contact>Institut Langevin &quot;Ondes et Images&quot; - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr</con